# Parte I: **Scraping de Datos de Vehículos en Milanuncios**

En esta etapa del proyecto, el objetivo principal es extraer información estructurada sobre vehículos disponibles en el sitio web de **Milanuncios** para realizar análisis posteriores. A continuación, se detalla el propósito, el enfoque técnico y la funcionalidad del proceso implementado:

---

## **1. Objetivo de la extracción**
El objetivo de esta etapa es realizar un scraping automatizado de datos de anuncios de vehículos de varias marcas (Toyota, BMW, Kia, Volkswagen, SEAT, Renault, Peugeot, Mercedes-Benz, Ford, Audi y Opel) publicados en Milanuncios. La información extraída permitirá analizar tendencias de precios, características de los vehículos y otros factores relevantes.

---

## **2. Estructura y lógica del scraping**
### **Fuentes de datos**
El scraping se realiza en la sección de anuncios clasificados de Milanuncios utilizando la URL base que permite buscar anuncios por marca y paginar los resultados.

- **URL base**:
  ```
  https://www.milanuncios.com/anuncios/?fromSearch=1&orden=relevance&s=<marca>&pagina=<número_página>
  ```
  - `<marca>`: Nombre de la marca que se itera en cada ejecución.
  - `<número_página>`: Número de la página a scrapear, limitado a las primeras 200 páginas por marca.

### **Información extraída**
Para cada anuncio válido, el scraper recopila los siguientes datos:
1. **Título**: Título principal del anuncio.
2. **Título extendido**: Detalle adicional proporcionado en el atributo `title` del enlace.
3. **URL del anuncio**: Enlace directo al detalle del anuncio.
4. **Número de fotos**: Número de imágenes asociadas al anuncio.
5. **Estado destacado**: Indica si el anuncio está marcado como "Destacado" en el sitio.
6. **Precio contado**: Precio del vehículo en modalidad de pago único.
7. **Precio financiado**: Precio del vehículo en modalidad de pago financiado (si aplica).
8. **Ubicación**: Ubicación del vendedor o punto de oferta.
9. **Kilometraje**: Distancia recorrida por el vehículo.
10. **Año**: Año de fabricación del vehículo.
11. **Tipo de combustible**: Tipo de combustible utilizado por el vehículo (diésel, gasolina, etc.).
12. **Descripción**: Breve texto descriptivo del anuncio.

### **Estrategia de iteración**
1. **Por marcas**: El proceso itera sobre una lista predefinida de marcas.
2. **Por páginas**: Para cada marca, se recorren hasta 200 páginas (o hasta que no haya más resultados disponibles).

### **Validación de datos**
- Los datos son validados durante el scraping para evitar errores como `NoneType` (valores inexistentes). Si no se encuentra un dato en particular, se asigna el valor "No disponible".

### **Manejo del servidor**
- Se emplean encabezados de solicitud (`headers`) para simular un navegador y evitar bloqueos del servidor.
- Se agrega una pausa de 2 segundos entre solicitudes para respetar los recursos del servidor y evitar ser bloqueado.

---

## **3. Funcionalidad técnica**
El código utiliza las siguientes herramientas y técnicas:

1. **Librerías utilizadas**:
   - `requests`: Para enviar solicitudes HTTP a las páginas de interés.
   - `BeautifulSoup`: Para analizar el HTML y extraer los datos específicos mediante selectores de etiquetas y clases CSS.
   - `pandas`: Para estructurar los datos extraídos en un formato tabular (DataFrame) y exportarlos como un archivo CSV.
   - `time`: Para implementar pausas entre solicitudes.

2. **Gestión de errores**:
   - Si una página no está disponible (`response.status_code` diferente de 200), el proceso para esa marca se detiene y se pasa a la siguiente.

3. **Almacenamiento**:
   - Todos los datos se almacenan en una lista de diccionarios durante el scraping y, al final, se consolidan en un DataFrame.
   - El DataFrame se exporta como un archivo `CSV` con el nombre `milanuncios_cars_data.csv`.

---

## **4. Salida**
La ejecución del script genera un archivo CSV que contiene los datos estructurados. El archivo incluye:
- **Total de registros**: Número de anuncios scrapeados para todas las marcas.
- **Columnas**: Nombre de la marca, título, precio, ubicación, descripción, entre otros atributos.

Ejemplo de estructura del CSV:

| Brand        | Title               | Price_Contado | Location  | Year | Kilometers | Fuel_Type | Description          |
|--------------|---------------------|---------------|-----------|------|------------|-----------|----------------------|
| BMW          | BMW X5 3.0d         | 35,000 €      | Madrid    | 2018 | 50,000 km  | Diésel    | Coche en perfecto estado. |
| Toyota       | Toyota Yaris Hybrid | 20,500 €      | Barcelona | 2020 | 10,000 km  | Híbrido   | Único dueño, revisión al día. |

---



In [ ]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Lista de marcas
brands = ["Toyota", "BMW", "Kia", "Volkswagen", "SEAT", "Renault", "Peugeot",
          "Mercedes-Benz", "Ford", "Audi", "Opel"]


base_url = "https://www.milanuncios.com/anuncios/?fromSearch=1&orden=relevance&s={}&pagina={}"


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
}


all_data = []


for brand in tqdm(brands, desc="Marcas", unit="marca"):
    print(f"Scrapeando datos para la marca: {brand}...")


    for page in tqdm(range(1, 201), desc=f"Páginas para {brand}", unit="página", leave=False):
        response = requests.get(base_url.format(brand, page), headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")


            elements = soup.find_all("article")


            new_records = 0

            for element in elements:
                # Filtrar solo anuncios válidos
                if "ma-AdCardV2" in element.get("class", []):
                    try:

                        title = element.find("h2", class_="ma-SharedText--m").text.strip() if element.find("h2", class_="ma-SharedText--m") else "No disponible"

                        link_element = element.find("a", class_="ma-AdCardListingV2-TitleLink")
                        link = link_element["href"] if link_element and "href" in link_element.attrs else "No disponible"
                        full_link = f"https://www.milanuncios.com{link}" if link != "No disponible" else "No disponible"

                        extended_title = element.find("a", class_="ma-AdCardListingV2-TitleLink")["title"] if element.find("a", class_="ma-AdCardListingV2-TitleLink") else "No disponible"

                        photo_count = element.find("figcaption", class_="ma-AdCardV2-photoCaption").text.strip() if element.find("figcaption", class_="ma-AdCardV2-photoCaption") else "No disponible"

                        is_featured = "Destacado" if element.find("div", class_="ma-AdCardV2-headerListing-caption--highlighted") else "No"

                        price_contado = element.find("span", class_="ma-AdPrice-value--dark").text.strip() if element.find("span", class_="ma-AdPrice-value--dark") else "No disponible"

                        price_financiado = element.find("span", class_="ma-AdPrice-value--gray").text.strip() if element.find("span", class_="ma-AdPrice-value--gray") else "No disponible"

                        location = element.find("span", class_="ma-AdLocation-text").text.strip() if element.find("span", class_="ma-AdLocation-text") else "No disponible"


                        tags = element.find_all("li", class_="ma-AdTagList-item")
                        kilometers = tags[0].text.strip() if len(tags) > 0 else "No disponible"
                        year = tags[1].text.strip() if len(tags) > 1 else "No disponible"
                        fuel_type = tags[2].text.strip() if len(tags) > 2 else "No disponible"


                        description = element.find("p", class_="ma-AdCardV2-description").text.strip() if element.find("p", class_="ma-AdCardV2-description") else "No disponible"


                        all_data.append({
                            "Brand": brand,
                            "Title": title,
                            "Extended_Title": extended_title,
                            "URL": full_link,
                            "Photo_Count": photo_count,
                            "Featured": is_featured,
                            "Price_Contado": price_contado,
                            "Price_Financiado": price_financiado,
                            "Location": location,
                            "Kilometers": kilometers,
                            "Year": year,
                            "Fuel_Type": fuel_type,
                            "Description": description,
                        })

                        new_records += 1
                    except AttributeError:
                        continue

            # Pausa para evitar sobrecarga en el servidor
            time.sleep(2)
        else:
            print(f"Error al acceder a la página {page} para la marca {brand}: Código {response.status_code}")
            break


df = pd.DataFrame(all_data)


print("Proceso completado. Datos guardados en 'milanuncios_cars_data.csv'.")


Marcas:   0%|          | 0/11 [00:00<?, ?marca/s]

Scrapeando datos para la marca: Toyota...



Marcas:   9%|▉         | 1/11 [10:43<1:47:14, 643.49s/marca]

Scrapeando datos para la marca: BMW...



Marcas:  18%|█▊        | 2/11 [22:08<1:40:08, 667.66s/marca]

...

Scrapeando datos para la marca: Audi...



Marcas:  91%|█████████ | 10/11 [1:52:43<11:12, 672.72s/marca]

Scrapeando datos para la marca: Opel...



Marcas: 100%|██████████| 11/11 [2:03:39<00:00, 674.54s/marca]

Proceso completado. Datos guardados en 'milanuncios_cars_data.csv'.


In [ ]:
df

,Brand,Title,Extended_Title,URL,Photo_Count,Featured,Price_Contado,Price_Financiado,Location,Kilometers,Year,Fuel_Type,Description
0,Toyota,Toyota - Auris,Toyota - Auris,https://www.milanuncios.com/toyota-de-segunda-...,9,Destacado,15.500 €,No disponible,Corralejo (Las Palmas),66.000 kms,2019,híbrido,"Vendo Toyota auris Híbrido, precio más bajo de..."
1,Toyota,TOYOTA - GR86,TOYOTA - GR86,https://www.milanuncios.com/toyota-de-segunda-...,26,Destacado,46.900 €,44.900 €,Pinto (Madrid),8.900 kms,2024,gasolina,Espectacular unidad de toyota gr86 2.4 235cv r...
2,Toyota,TOYOTA - GR86,TOYOTA - GR86,https://www.milanuncios.com/toyota-de-segunda-...,33,Destacado,43.990 €,41.990 €,Pinto (Madrid),6.990 kms,2023,gasolina,Espectacular unidad de toyota gr86 2.4 235cv r...
3,Toyota,TOYOTA - Priusplus Advance,TOYOTA - Priusplus Advance,No disponible,31,Destacado,23.500 €,23.500 €,Puerto Real (Cádiz),63.884 kms,2021,híbrido,
4,Toyota,TOYOTA - GR Yaris,TOYOTA - GR Yaris,https://www.milanuncios.com/toyota-de-segunda-...,18,Destacado,33.480 €,No disponible,Oviedo (Asturias),49.650 kms,2020,gasolina,Precio al contado: 33.480€\nprecio alternativo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10985,Opel,Opel - Movano,Opel - Movano,No disponible,No disponible,No,4.000 €,No disponible,Carrascal de Barregas (Salamanca),285.000 kms,1999,diesel,Se vende open movano + Renault máster para pie...
10986,Opel,RETROVISOR Opel,RETROVISOR Opel,https://www.milanuncios.com/carroceria/retrovi...,No disponible,No,No disponible,No disponible,Mataro (Barcelona),No disponible,No disponible,No disponible,Pieza Nº 316431. Retrovisor Electrico Izquierd...
10987,Opel,RETROVISOR Opel,RETROVISOR Opel,https://www.milanuncios.com/carroceria/retrovi...,No disponible,No,54 €,No disponible,Mataro (Barcelona),No disponible,No disponible,No disponible,Pieza Nº 315958. Retrovisor Derecho Opel. Año:...
10988,Opel,OPEL zafira a,OPEL zafira a,https://www.milanuncios.com/otros-recambios/op...,No disponible,No,No disponible,No disponible,Bolaños de Calatrava (Ciudad Real),No disponible,No disponible,No disponible,Pieza nº 134572. opel zafira a 2.0 16v di (82 ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Guarda el DataFrame directamente en tu Google Drive
df.to_csv("/content/drive/My Drive/milanuncios_cars_data.csv", index=True)


In [ ]:
df.to_pickle("/content/drive/My Drive/milanuncios_cars_data.pkl")


In [ ]:
df.to_parquet("/content/drive/My Drive/milanuncios_cars_data.parquet", index=True)
